In [ ]:
#%jsroot on -> works for ROOT notebooks only :(

import ROOT 
import pyRratio as pyr

pyr._sin_sq_theta_W = 0.23

a) We will plot the different components of the $R$-ratio, defined as, 

$$ R = \dfrac{\sigma(e^+e^- \rightarrow \mathrm{hadrons})}{\sigma(e^+e^- \rightarrow \mathrm{\mu^+\mu^-})}$$

and compare to real data in the end to check our derivation corresponds to nature! The data are stored in the file `rpp2012-hadronicrpp_page1001.dat.txt`

In [ ]:
from ROOT import TGraph

graph_m = TGraph()
graph_m_qed = TGraph()
graph_i = TGraph()
graph_i_full = TGraph()

graph_h = TGraph()
graph_r = TGraph()
graph_r_full = TGraph()

graph_afb = TGraph()

from pyDatasetForRratio import readFile

graph_data = readFile( fname = 'rpp2012-hadronicrpp_page1001.dat.txt', maxenergy = 160.0 )

The functions for each component of the differential decay rate are defined in `pyRratio.py`.

In [ ]:
sqrts = 0.3
point = 0

while ( sqrts < 160.0 ):
    
    rate_m_full = pyr.differential_decay_rate( sqrts**2 , pyr.electron , [pyr.muon] )
    rate_m_qed  = pyr.differential_decay_rate_qed( sqrts**2 )
    
    rate_h      = pyr.differential_decay_rate( sqrts**2 , pyr.electron , pyr.quarks, NC = 3. )

    rate_i      = pyr.gamma_Z_interference( sqrts**2, pyr.electron, pyr.quarks, NC = 3. )
    
    afb         = pyr.forward_backward_asymmetry( sqrts**2, pyr.electron, pyr.muon )
    
    graph_afb.SetPoint( graph_afb.GetN(), sqrts, afb )
    
    if ( rate_h > 0 and rate_m_qed > 0 and rate_m_full>0 ):
        
        graph_m.SetPoint( point, sqrts, rate_m_full )
        graph_m_qed.SetPoint( point, sqrts, rate_m_qed )
        graph_h.SetPoint( point, sqrts, rate_h ) 
        
        graph_i.SetPoint( point, sqrts, rate_i/rate_m_qed )
        graph_i_full.SetPoint( point, sqrts, rate_i/rate_m_full)

        graph_r.SetPoint( point, sqrts, rate_h/rate_m_qed )
        graph_r_full.SetPoint( point, sqrts, rate_h/rate_m_full )  

    
    sqrts += 0.2
    point += 1


from ROOT import gPad, TCanvas

can_r = TCanvas('can_r')
can_r.SetLogx() # Set logx to see steps in ratio 
gPad.SetLogy()

graph_r.Draw('AL')
graph_r.GetYaxis().SetRangeUser(0.1,100000)

graph_r.SetLineColor( ROOT.kRed )
graph_r_full.SetLineColor( ROOT.kBlue )

graph_r.GetXaxis().SetTitle( '#sqrt{s} [GeV]' )
graph_r.GetYaxis().SetTitle( '#it{R}' )
graph_r_full.Draw('L')
graph_data.Draw('P+')


can_i = TCanvas('can_i')
graph_i.Draw('AL')
graph_i.SetLineColor( ROOT.kRed )
graph_i_full.SetLineColor( ROOT.kBlue )

graph_i.GetXaxis().SetTitle( '#sqrt{s} [GeV]' )
graph_i.GetYaxis().SetTitle( 'Interference term' )
graph_i_full.Draw('L')


In [ ]:
can_r.Draw()

In [ ]:
can_i.Draw()

b) Change the value of the weak-mixing angle $\theta_{W}$ at the top of this notebook to see how the shape of $R$ changes. How does this compare now do the data?

Now let's compare the interference with different values of $\theta_W$ in one go. We have some functions to make the plots easily for us (defined in `pyRratio.py`).

We can also plot the forward-backward asymmetry - note this is what the experiments actually measure to access $sin(\theta_W)$! This is typically calculated using the leptonic cross-sections.

$$A_{FB}=\dfrac{\sigma_F- \sigma_B}{\sigma_F+\sigma_B}$$

where the forward ($F$) and backward $B$ directions are defined in the COM frame (i.e $cos(\theta)$ > 0 or < 0 ). 

In [ ]:
pyr._sin_sq_theta_W = 0.23
i23 = pyr.interference_with_sqrts( pyr.electron, pyr.quarks, NC = 3. ); i23.SetLineColor(ROOT.kRed)
a23 = pyr.afb_with_sqrts( pyr.electron, pyr.muon ); i23.SetLineColor(ROOT.kRed)

pyr._sin_sq_theta_W = 0.3
i3 = pyr.interference_with_sqrts( pyr.electron, pyr.quarks, NC = 3. ); i3.SetLineColor(ROOT.kGreen+1)
a3 = pyr.afb_with_sqrts( pyr.electron, pyr.muon ); a3.SetLineColor(ROOT.kGreen+1)

c_afb = TCanvas("c_afb","",800,400)
c_afb.Divide(2)

c_afb.cd(1)
i23.GetYaxis().SetTitle("Interference term")
i23.GetXaxis().SetTitle( '#sqrt{s} [GeV]' )
i23.Draw("AL")
i3.Draw("L")

c_afb.cd(2)
a23.GetYaxis().SetTitle("A_{FB}")
a23.GetXaxis().SetTitle( '#sqrt{s} [GeV]' )
a23.Draw("AL")
a3.Draw("L")

c_afb.Draw()